In [ ]:
# EX1: DATABASE DEFINITION LANGUAGE (SQL Server) USING pyodbc - CSV file TO DATABASE 

# The following code emphasizes pyodbc functionality and how it can be used to create a connection with a database
# The data in this file relates primarily to branch level transactions

import pyodbc
import pandas as pd

create_query = '''--Creating the table given that it does not exist
                  IF Object_ID('sandboxtest') IS NULL

                  CREATE TABLE [sandbox].[dbo].[sandboxtest]
                  (
                      [Branch_Name] VARCHAR(15) NOT NULL,
                      [Account_Deposit_Category_Desc] INTEGER NOT NULL,
                      [Txn_Amount] INTEGER NOT NULL,
                      [Txn_Total_Number_of_Transactions] INTEGER NOT NULL,
                  );'''

DRIVER = '{ODBC Driver 17 for SQL Server}'
SERVER = 'RMT-LT-W10-CM\SQLSERVER'
DATABASE = 'sandbox'

#Define Connection
connection_string = '''
Driver={driver};
Server={server};
Database={database};
Trusted_Connection=yes;
'''.format(
    driver=DRIVER,
    server=SERVER,
    database=DATABASE
)

#Connect to Database
connection_object: pyodbc.Connection = pyodbc.connect(connection_string)
cursor_object = pyodbc.Cursor = connection_object.cursor()

#Import file as pandas dataframe to be imported to dataframe
sandbox_df: pd.DataFrame = pd.read_csv(r"C:\Users\cmarshall\Desktop\sandbox.csv", encoding='latin-1')
sandbox_df.dtypes

#Insert Query to take pandas dataframe and insert into database
insert_query = '''
INSERT INTO [sandbox].[dbo].[sandboxtest]
(
    [Branch_Name],
    [Account_Deposit_Category_Desc],
    [Txn_Amount],
    [Txn_Total_Number_of_Transactions]
)
VALUES
(
    ?,?,?,?,?
)
'''
#Drop table so that records are not duplicated, this is necessary!!!
drop_table = '''DROP TABLE IF EXISTS dbo.sandboxtest'''

cursor_object.execute(drop_table)
cursor_object.commit()

#Create the table in the database
cursor_object.execute(create_query)
cursor_object.commit()

#Convert dataframe into record set (to list)
df_rec = sandbox_df.values.tolist()

#Execute the insert query
cursor_object.executemany(insert_query, df_rec)
cursor_object.commit()


basicSelect = '''SELECT * FROM [sandbox].[dbo].[sandboxtest];'''
df = cursor_object.execute(basicSelect).fetchall()
columns = [column[0] for column in cursor_object.description]
df = pd.DataFrame.from_records(data=df, columns=columns)
df.head()


basicSelect = \
'''
SELECT Branch_Name, Account_Deposit_Category_Desc, 
ROUND(SUM(Txn_Amount),2) AS Sum_Txn_Amount, 
ROUND(AVG(Txn_Amount),2) AS Avg_Txn_Amount, 
SUM(Txn_Total_Number_of_Transactions) AS Total_Txns
FROM transactions
WHERE Branch_Name IS NOT NULL AND Account_Deposit_Category_Desc IS NOT NULL AND Account_Deposit_Category_Desc != 'Revolving Credit'
GROUP BY Branch_Name, Account_Deposit_Category_Desc
ORDER BY Branch_Name, Avg_Txn_Amount DESC
'''
df = cursor_object.execute(basicSelect).fetchall()
columns = [column[0] for column in cursor_object.description]
df = pd.DataFrame.from_records(data=df, columns=columns)
df.rename(columns={"Branch_Name":'Branch Name', "Account_Deposit_Category_Desc":"Deposit Category"},inplace=True)
df